In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from sklearn.pipeline import FeatureUnion

In [2]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn

In [3]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb



In [46]:
from collections import Counter, OrderedDict
import collections

In [5]:
import joblib


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [7]:
import re

In [8]:
from textblob import TextBlob

Based on https://www.kaggle.com/venomsnake/analyzing-sarcasm-comprehensive-eda-detection and https://github.com/surajr/SarcasmDetection

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
df_isarcasm=pd.read_csv("/content/drive/MyDrive/ISarcasm/DataSet/train.En.csv")

In [65]:
df_isarcasm=df_isarcasm[['tweet','sarcastic']]

In [66]:
train, validate, test = \
              np.split(df_isarcasm.sample(frac=1, random_state=42), 
                       [int(.6*len(df_isarcasm)), int(.8*len(df_isarcasm))])

In [67]:
train_new=pd.concat([train, validate], ignore_index=True)

In [137]:
def count_character_type(text):
    text=str(text)

    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "!":
            specialChar+=1
        
    return specialChar
    
def count_question_mark(text):
    text=str(text)
    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "?":
            specialChar+=1
        
    return specialChar

The Python NLP library, TextBlob has a method to roughly quantify if a sentence is fact or opinion.¶
The method outputs a number ranging from 0 to 1, an output close to 0 indicates the sentence is highly factual and close to 1 means the sentence is highly subjective. Here, we take the sum for every comment, an overall higher sum then, will be indicative of higher subjectivity

In [141]:
def get_subjectivity(text):
    return TextBlob(str(text)).sentiment.subjectivity

The distribution of the 4 most important parts-of-speech (Noun, Verb, Adjective,Pronoun) in sarcastic and non sarcastic comments might shed some light, so let's plot it as a grouped bar chart

In [16]:
import spacy

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [18]:
def pos_counter(doc):
    verb_count = 0
    adj_count = 0
    pron_count = 0
    noun_count=0
    for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
        elif tok.pos_ == "ADJ":
            adj_count=adj_count+1
        elif tok.pos_ == "PRON":
            pron_count=pron_count+1
        elif tok.pos_ == "PROPN":
            noun_count=noun_count+1
    return (verb_count,adj_count,pron_count,noun_count)

In [144]:
def get_verb_count(text):
  verb_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
  return verb_count

def get_noun_count(text):
  noun_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PROPN":
            noun_count=noun_count+1
  return noun_count

def get_pron_count(text):
  pron_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PRON":
            pron_count=pron_count+1
  return pron_count


def get_adj_count(text):
  adj_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "ADJ":
            adj_count=adj_count+1
  return adj_count

Sarcastic comments tend to have profanity interspersed within, let's get a count of both sarcastic and non sarcastic and see how they compare

In [22]:
!pip install better_profanity 
from better_profanity import profanity

     |████████████████████████████████| 46 kB 2.6 MB/s 


In [145]:
def count_profane_words(text):
    count = 0
    for sent in str(text).split():
        if profanity.contains_profanity(sent) == True:
            count = count+1
    return count

get entity cloud

In [150]:
def entity_wordcloud(text):
    named_entities = []    
    sent = nlp(str(text))
    for ent in sent.ents:
        if ent.label_ == 'PERSON' or 'ORG' or 'GPE':
            named_entities.append(ent.text)
            
    doc = " ".join(named_entities)
    return doc

In [25]:
!git clone https://github.com/larsmans/sentiwordnet.git

Cloning into 'sentiwordnet'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [39]:
import csv

In [49]:
class load_senti_word_net(object):
    """
    constructor to load the file and read the file as CSV
    6 columns - pos, ID, PosScore, NegScore, synsetTerms, gloss
    synsetTerms can have multiple similar words like abducting#1 abducent#1 and will read each one and calculaye the scores
    """
    
    def __init__(self):
        sent_scores = collections.defaultdict(list)
        with io.open("/content/sentiwordnet/SentiWordNet_3.0.0_20130122.txt") as fname:
            file_content = csv.reader(fname, delimiter='\t',quotechar='"')
            
            for line in file_content:                
                if line[0].startswith('#') :
                    continue                    
                pos, ID, PosScore, NegScore, synsetTerms, gloss = line
                for terms in synsetTerms.split(" "):
                    term = terms.split("#")[0]
                    term = term.replace("-","").replace("_","")
                    key = "%s/%s"%(pos,term.split("#")[0])
                    try:
                        sent_scores[key].append((float(PosScore),float(NegScore)))
                    except:
                        sent_scores[key].append((0,0))
                    
        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value,axis=0)
        
        self.sent_scores = sent_scores    
     
    """
    For a word,
    nltk.pos_tag(["Suraj"])
    [('Suraj', 'NN')]
    """
    
    def score_word(self, word):
        pos = nltk.pos_tag([word])[0][1]
        return self.score(word, pos)
    
    def score(self,word, pos):
        """
        Identify the type of POS, get the score from the senti_scores and return the score
        """
        
        if pos[0:2] == 'NN':
            pos_type = 'n'
        elif pos[0:2] == 'JJ':
            pos_type = 'a'
        elif pos[0:2] =='VB':
            pos_type='v'
        elif pos[0:2] =='RB':
            pos_type = 'r'
        else:
            pos_type =  0
            
        if pos_type != 0 :    
            loc = pos_type+'/'+word
            score = self.sent_scores[loc]
            if len(score)>1:
                return score
            else:
                return np.array([0.0,0.0])
        else:
            return np.array([0.0,0.0])
        
    """
    Repeat the same for a sentence
    nltk.pos_tag(word_tokenize("My name is Suraj"))
    [('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Suraj', 'NNP')]    
    """    
        
    def score_sentencce(self, sentence):
        pos = nltk.pos_tag(sentence)
        # print (pos)
        mean_score = np.array([0.0, 0.0])
        for i in range(len(pos)):
            mean_score += self.score(pos[i][0], pos[i][1])
            
        return mean_score
    
    def pos_vector(self, sentence):
        pos_tag = nltk.pos_tag(sentence)
        vector = np.zeros(4)
        
        for i in range(0, len(pos_tag)):
            pos = pos_tag[i][1]
            if pos[0:2]=='NN':
                vector[0] += 1
            elif pos[0:2] =='JJ':
                vector[1] += 1
            elif pos[0:2] =='VB':
                vector[2] += 1
            elif pos[0:2] == 'RB':
                vector[3] += 1
                
        return vector
            

In [37]:
def gram_features(features,sentence):
    sentence_rep = replace_reg(str(sentence))
    token = nltk.word_tokenize(sentence_rep)
    token = [porter.stem(i.lower()) for i in token]        
    
    bigrams = nltk.bigrams(token)
    bigrams = [tup[0] + ' ' + tup[1] for tup in bigrams]
    grams = token + bigrams
    #print (grams)
    for t in grams:
        features['contains(%s)'%t]=1.0

In [38]:
def pos_features(features,sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)
    token = [ porter.stem(each.lower()) for each in token]
    pos_vector = sentiments.pos_vector(token)
    for j in range(len(pos_vector)):
        features['POS_'+str(j+1)] = pos_vector[j]
    # print ("done")
    

In [29]:
def capitalization(features,sentence):
    count = 0
    for i in range(len(sentence)):
        count += int(sentence[i].isupper())
    features['Capitalization'] = int(count > 3)
    # print (count)

In [53]:
import nltk
import re

#dictionnary to sentiment analysis
emo_repl = {
    #good emotions
    "&lt;3" : " good ",
    ":d" : " good ",
    ":dd" : " good ",
    ":p" : " good ",
    "8)" : " good ",
    ":-)" : " good ",
    ":)" : " good ",
    ";)" : " good ",
    "(-:" : " good ",
    "(:" : " good ",
    
    "yay!" : " good ",
    "yay" : " good ",
    "yaay" : " good ",
    "yaaay" : " good ",
    "yaaaay" : " good ",
    "yaaaaay" : " good ",    
    #bad emotions
    ":/" : " bad ",
    ":&gt;" : " sad ",
    ":')" : " sad ",
    ":-(" : " bad ",
    ":(" : " bad ",
    ":s" : " bad ",
    ":-s" : " bad "
}

#dictionnary for general (i.e. topic modeler)
emo_repl2 = {
    #good emotions
    "&lt;3" : " heart ",
    ":d" : " smile ",
    ":p" : " smile ",
    ":dd" : " smile ",
    "8)" : " smile ",
    ":-)" : " smile ",
    ":)" : " smile ",
    ";)" : " smile ",
    "(-:" : " smile ",
    "(:" : " smile ",
       
    #bad emotions
    ":/" : " worry ",
    ":&gt;" : " angry ",
    ":')" : " sad ",
    ":-(" : " sad ",
    ":(" : " sad ",
    ":s" : " sad ",
    ":-s" : " sad "
}

#general
re_repl = {
    r"\br\b" : "are",
    r"\bu\b" : "you",
    r"\bhaha\b" : "ha",
    r"\bhahaha\b" : "ha",
    r"\bdon't\b" : "do not",
    r"\bdoesn't\b" : "does not",
    r"\bdidn't\b" : "did not",
    r"\bhasn't\b" : "has not",
    r"\bhaven't\b" : "have not",
    r"\bhadn't\b" : "had not",
    r"\bwon't\b" : "will not",
    r"\bwouldn't\b" : "would not",
    r"\bcan't\b" : "can not",
    r"\bcannot\b" : "can not"    
}

emo_repl_order = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl.keys()]))]
emo_repl_order2 = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl2.keys()]))]

def replace_emo(sentence):
    sentence2 = sentence
    for k in emo_repl_order:
        sentence2 = sentence2.replace(k,emo_repl[k])
    for r, repl in re_repl.items():
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

def replace_reg(sentence):
    sentence2 = sentence
    for k in emo_repl_order2:
        sentence2 = sentence2.replace(k,emo_repl2[k])
    for r, repl in re_repl.items(): 
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

In [31]:
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

class topic(object):
    def __init__(self, nbtopic = 100, alpha=1,model=None,dicttp=None):
        self.nbtopic = nbtopic
        self.alpha = alpha
        self.porter = nltk.PorterStemmer()
        self.stop = stopwords.words('english')+['.','!','?','"','...','\\',"''",'[',']','~',"'m","'s",';',':','..','$']
        if model!=None and dicttp!=None:
            self.lda = models.ldamodel.LdaModel.load(model)
            self.dictionary =  corpora.Dictionary.load(dicttp)
            
    def fit(self,documents):
        
        documents_mod = documents
        tokens = [nltk.word_tokenize(sentence) for sentence in documents_mod]
        tokens = [[self.porter.stem(t.lower()) for t in sentence if t.lower() not in self.stop] for sentence in tokens]        
            
        self.dictionary = corpora.Dictionary(tokens)
        corpus = [self.dictionary.doc2bow(text) for text in tokens]
        self.lda = models.ldamodel.LdaModel(corpus,id2word=self.dictionary, num_topics=self.nbtopic,alpha=self.alpha)
        
        self.lda.save('topics.tp')
        self.dictionary.save('topics_dict.tp')
        
    def get_topic(self,topic_number):
        
        return self.lda.print_topic(topic_number)
    
    def transform(self,sentence):
        
        sentence_mod = sentence
        tokens = nltk.word_tokenize(sentence_mod)
        tokens = [self.porter.stem(t.lower()) for t in tokens if t.lower() not in self.stop] 
        corpus_sentence = self.dictionary.doc2bow(tokens)
        
        return self.lda[corpus_sentence] 

In [34]:
porter = nltk.PorterStemmer()


In [57]:
import string
def sentiment_extract(features, sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)    
    token = [porter.stem(i.lower()) for i in token]   
    mean_sentiment = sentiments.score_sentencce(token)
    features["Positive Sentiment"] = mean_sentiment[0]
    features["Negative Sentiment"] = mean_sentiment[1]
    features["sentiment"] = mean_sentiment[0] - mean_sentiment[1]
    #print(mean_sentiment[0], mean_sentiment[1])
    
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in token]).strip())
        features["Blob Polarity"] = text.sentiment.polarity
        features["Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["Blob Polarity"] = 0
        features["Blob Subjectivity"] = 0
        # print("do nothing")
        
    
    first_half = token[0:len(token)//2]    
    mean_sentiment_half = sentiments.score_sentencce(first_half)
    features["positive Sentiment first half"] = mean_sentiment_half[0]
    features["negative Sentiment first half"] = mean_sentiment_half[1]
    features["first half sentiment"] = mean_sentiment_half[0]-mean_sentiment_half[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in first_half]).strip())
        features["first half Blob Polarity"] = text.sentiment.polarity
        features["first half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["first Blob Polarity"] = 0
        features["first Blob Subjectivity"] = 0
        # print("do nothing")
    
    second_half = token[len(token)//2:]
    mean_sentiment_sechalf = sentiments.score_sentencce(second_half)
    features["positive Sentiment second half"] = mean_sentiment_sechalf[0]
    features["negative Sentiment second half"] = mean_sentiment_sechalf[1]
    features["second half sentiment"] = mean_sentiment_sechalf[0]-mean_sentiment_sechalf[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in second_half]).strip())
        features["second half Blob Polarity"] = text.sentiment.polarity
        features["second half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["second Blob Polarity"] = 0
        features["second Blob Subjectivity"] = 0
        # print("do nothing")  
    

In [41]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [42]:
topic_mod = topic(nbtopic=200,alpha='symmetric')
topic_mod.fit(train_new['tweet'].values)
# topic_mod = topic(model=os.path.join('topics.tp'),dicttp=os.path.join('topics_dict.tp'))
def topic_feature(features,sentence,topic_modeler):    
    topics = topic_modeler.transform(sentence)    
    for j in range(len(topics)):
        features['Topic :'] = topics[j][1]

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [101]:
!pip install emoji

     |████████████████████████████████| 170 kB 4.4 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=b609a270a241ecbf459fa74fa6033a76dde9320daaa81fb66fd98719aa569f9a
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [102]:
from emoji import UNICODE_EMOJI



In [134]:
"""
Function to extract the below features.
1. Presence of emoji's
2. Count of number of Question marks
3. Count of number of Exclamation marks
4. presence of hashtags other than #sarcasm
5. presence of any @user tweets

"""
from collections import defaultdict

def Emoji_present(text):
    emoji = {
    "&lt;3" : "positive",":D" : "positive",	":d" : "positive", ":dd" : "positive", ":P" : "positive", ":p" : "positive","8)" : "positive",
    "8-)" : "positive",  ":-)" : "positive",    ":)" : "positive",    ";)" : "positive",    "(-:" : "positive",    "(:" : "positive",
    ":')" : "positive",    "xD" : "positive",    "XD" : "positive",  "yay!" : "positive",  "yay" : "positive",  "yaay" : "positive",
    "yaaay" : "positive",  "yaaaay" : "positive", "yaaaaay" : "positive", "Yay!" : "positive", "Yay" : "positive", "Yaay" : "positive",
    "Yaaay" : "positive", "Yaaaay" : "positive", "Yaaaaay" : "positive",  ":/" : "negative", "&gt;" : "negative", ":'(" : "negative",
    ":-(" : "negative", ":(" : "negative", ":s" : "negative",":-s" : "negative","-_-" : "negative", "-.-" : "negative" }
    emoji_count =defaultdict(int) 
    isPresent=False    
        
    for word in str(text):
        if word in emoji:            
            isPresent = True

    
    count = 0
    for emoji in UNICODE_EMOJI:
        count += str(text).count(emoji)
        if count>=1:
          isPresent=True      

    
    return isPresent 



In [91]:
import io

sentiments = load_senti_word_net()

def get_features(sentence, topic_modeler):
    features = {}
    # gram_features(features,sentence)
    pos_features(features,sentence)
    sentiment_extract(features, sentence)
    capitalization(features,sentence)
    topic_feature(features, sentence,topic_modeler)
    return features

In [55]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [59]:
train_new.dropna(inplace=True)

In [81]:
train_new

,tweet,sarcastic
0,Less than 5 minutes after getting my phone bac...,0
1,Love seeing 15 year olds being vaccinated in t...,1
2,I get a lot of boy who cried wolf vibes from t...,1
3,im about to just walk into a place and start w...,1
4,Every time I comment on a TikTok asking for ca...,0
...,...,...
2769,This weekend I hit a delivery biker with a car...,0
2770,This country is screwed let’s be honest,0
2771,I always come up with some pretty interesting ...,0
2772,@andy_mansell72 @VicOtley @garry_whu @NiaAnts ...,1


In [96]:
import re
featureset=[]
for i in range(0,len(train_new)):
    # if i==2774or i==2775:
    #   continue
    temp = str(train_new["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    featureset.append((get_features(temp,topic_mod), train_new["sarcastic"][i], train_new["tweet"][i]))

In [97]:
c = []
for i in range(0,len(featureset)):
    c.append(pd.DataFrame(featureset[i][0],index=[i]))
result = pd.concat(c)

In [98]:
result.insert(loc=0,column="label",value='0')
for i in range(0, len(featureset)):
    result["label"].loc[i] = featureset[i][1]   

result.insert(loc=0,column="text",value='0')
for i in range(0, len(featureset)):
    result["text"].loc[i] = featureset[i][2]  
    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [99]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,Less than 5 minutes after getting my phone bac...,0,11.0,2.0,2.0,1.0,0.095620,0.432559,-0.336939,-0.083333,0.033333,0.076389,0.315972,-0.239583,-0.083333,0.033333,0.019231,0.116587,-0.097356,0.00,0.000000,0,0.107272
1,Love seeing 15 year olds being vaccinated in t...,1,7.0,2.0,6.0,0.0,0.980875,0.337487,0.643387,0.283333,0.377778,0.856971,0.159856,0.697115,0.300000,0.400000,0.123904,0.177632,-0.053728,0.25,0.333333,0,0.072137
2,I get a lot of boy who cried wolf vibes from t...,1,8.0,2.0,3.0,0.0,0.380556,0.743651,-0.363095,0.000000,0.000000,0.097222,0.251984,-0.154762,0.000000,0.000000,0.283333,0.491667,-0.208333,0.00,0.000000,0,0.150040
3,im about to just walk into a place and start w...,1,6.0,0.0,3.0,3.0,0.213412,0.136337,0.077076,0.000000,0.000000,0.157961,0.039062,0.118899,0.000000,0.000000,0.055451,0.097274,-0.041823,0.00,0.000000,0,0.175058
4,Every time I comment on a TikTok asking for ca...,0,15.0,3.0,1.0,0.0,0.717014,0.522222,0.194792,0.000000,0.100000,0.458333,0.466667,-0.008333,0.000000,0.100000,0.258681,0.055556,0.203125,0.00,0.000000,1,0.252168


In [104]:
result['contain_emoji']=result['text'].apply(Emoji_present)

In [105]:
result['count_special_chracter']=result['text'].apply(count_character_type)

In [106]:
result['count_question_marks']=result['text'].apply(count_question_mark)

In [107]:
result['subjectivity']=result['text'].apply(get_subjectivity)

In [108]:
result['count_verbs']=result['text'].apply(get_verb_count)

In [109]:
result['count_nouns']=result['text'].apply(get_noun_count)

In [110]:
result['count_pronun']=result['text'].apply(get_pron_count)

In [111]:
result['count_adjct']=result['text'].apply(get_adj_count)

In [112]:
result['entities']=result['text'].apply(entity_wordcloud)

In [113]:
result['count_profane']=result['text'].apply(count_profane_words)

In [114]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane
0,Less than 5 minutes after getting my phone bac...,0,11.0,2.0,2.0,1.0,0.095620,0.432559,-0.336939,-0.083333,0.033333,0.076389,0.315972,-0.239583,-0.083333,0.033333,0.019231,0.116587,-0.097356,0.00,0.000000,0,0.107272,True,0,0,0.033333,2,0,1,1,Less than 5 minutes,0
1,Love seeing 15 year olds being vaccinated in t...,1,7.0,2.0,6.0,0.0,0.980875,0.337487,0.643387,0.283333,0.377778,0.856971,0.159856,0.697115,0.300000,0.400000,0.123904,0.177632,-0.053728,0.25,0.333333,0,0.072137,True,0,0,0.466667,3,1,1,1,15 year olds the end of June first,0
2,I get a lot of boy who cried wolf vibes from t...,1,8.0,2.0,3.0,0.0,0.380556,0.743651,-0.363095,0.000000,0.000000,0.097222,0.251984,-0.154762,0.000000,0.000000,0.283333,0.491667,-0.208333,0.00,0.000000,0,0.150040,True,0,0,0.000000,4,2,2,0,wolf vibes the Red Cross,0
3,im about to just walk into a place and start w...,1,6.0,0.0,3.0,3.0,0.213412,0.136337,0.077076,0.000000,0.000000,0.157961,0.039062,0.118899,0.000000,0.000000,0.055451,0.097274,-0.041823,0.00,0.000000,0,0.175058,True,0,0,0.000000,6,0,3,0,,0
4,Every time I comment on a TikTok asking for ca...,0,15.0,3.0,1.0,0.0,0.717014,0.522222,0.194792,0.000000,0.100000,0.458333,0.466667,-0.008333,0.000000,0.100000,0.258681,0.055556,0.203125,0.00,0.000000,1,0.252168,True,0,0,0.533333,6,4,3,2,TikTok,0


In [117]:
len(test)

694

In [126]:
import re
feature_set_test=[]
for i, row in test.iterrows():
    # if i==0:
    #   continue
    temp = str(test["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    feature_set_test.append((get_features(temp,topic_mod), test["sarcastic"][i],test['tweet'][i]))

In [127]:
c = []
for i in range(0,len(feature_set_test)):
    c.append(pd.DataFrame(feature_set_test[i][0],index=[i]))
result_test = pd.concat(c)

In [128]:
result_test.insert(loc=0,column="label",value='0')
for i in range(0, len(feature_set_test)):
    result_test["label"].loc[i] = feature_set_test[i][1]   

result_test.insert(loc=0,column="text",value='0')
for i in range(0, len(feature_set_test)):
    result_test["text"].loc[i] = feature_set_test[i][2]   
    
    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [129]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,The world according to Jeff Goldblum is such a...,0,4.0,1.0,2.0,0.0,0.322917,0.354167,-0.031250,0.000000,0.500000,0.156250,0.187500,-0.031250,0.00,0.0,0.166667,0.166667,0.000000,0.000000,0.500000,0,0.251250
1,@kevinabstract ye,0,2.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.751250
2,Rubbing my tears into my face is part of my sk...,1,6.0,0.0,2.0,0.0,0.019231,0.184295,-0.165064,0.800000,0.700000,0.019231,0.184295,-0.165064,0.00,0.0,0.000000,0.000000,0.000000,0.800000,0.700000,0,0.143571
3,What's the German word for when you start a po...,0,10.0,5.0,7.0,3.0,0.922618,0.699750,0.222868,0.134091,0.188636,0.216518,0.158482,0.058036,0.05,0.1,0.706100,0.541268,0.164833,0.218182,0.277273,0,0.064480
4,"Prayers go up, blessings come down",0,2.0,0.0,2.0,0.0,0.077381,0.010119,0.067262,-0.155556,0.288889,0.041667,0.004167,0.037500,0.00,0.0,0.035714,0.005952,0.029762,-0.155556,0.288889,0,0.173570


In [135]:
result_test['contain_emoji']=result_test['text'].apply(Emoji_present)

In [138]:
result_test['count_special_chracter']=result_test['text'].apply(count_character_type)

In [139]:
result_test['count_question_marks']=result_test['text'].apply(count_question_mark)

In [142]:
result_test['subjectivity']=result_test['text'].apply(get_subjectivity)

In [146]:
result_test['count_verbs']=result_test['text'].apply(get_verb_count)

In [147]:
result_test['count_nouns']=result_test['text'].apply(get_noun_count)

In [148]:
result_test['count_pronun']=result_test['text'].apply(get_pron_count)

In [149]:
result_test['count_adjct']=result_test['text'].apply(get_adj_count)

In [151]:
result_test['entities']=result_test['text'].apply(entity_wordcloud)

In [152]:
result_test['count_profane']=result_test['text'].apply(count_profane_words)

In [153]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane
0,The world according to Jeff Goldblum is such a...,0,4.0,1.0,2.0,0.0,0.322917,0.354167,-0.031250,0.000000,0.500000,0.156250,0.187500,-0.031250,0.00,0.0,0.166667,0.166667,0.000000,0.000000,0.500000,0,0.251250,False,0,0,0.500000,1,2,0,0,Jeff Goldblum,0
1,@kevinabstract ye,0,2.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.751250,False,0,0,0.000000,0,2,0,0,,0
2,Rubbing my tears into my face is part of my sk...,1,6.0,0.0,2.0,0.0,0.019231,0.184295,-0.165064,0.800000,0.700000,0.019231,0.184295,-0.165064,0.00,0.0,0.000000,0.000000,0.000000,0.800000,0.700000,0,0.143571,False,0,0,0.700000,1,0,0,0,,0
3,What's the German word for when you start a po...,0,10.0,5.0,7.0,3.0,0.922618,0.699750,0.222868,0.134091,0.188636,0.216518,0.158482,0.058036,0.05,0.1,0.706100,0.541268,0.164833,0.218182,0.277273,0,0.064480,True,0,1,0.188636,4,0,5,4,German 2 years old present-day every week 3 mo...,0
4,"Prayers go up, blessings come down",0,2.0,0.0,2.0,0.0,0.077381,0.010119,0.067262,-0.155556,0.288889,0.041667,0.004167,0.037500,0.00,0.0,0.035714,0.005952,0.029762,-0.155556,0.288889,0,0.173570,True,0,0,0.288889,2,0,0,0,,0


In [154]:
result_test.columns

Index(['text', 'label', 'POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'entities', 'count_profane'],
      dtype='object')

In [161]:
result_test.dropna(inplace=True)

In [162]:
result.dropna(inplace=True)

In [163]:
x_train,y_train=result[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']],result[['label']]

In [164]:
x_test,y_test=result_test[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']],result_test[['label']]

In [157]:
def print_statistics(y, y_pred):
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred))
    return accuracy, precision, recall, f_score



def plot_coefficients(classifier, feature_names, top_features=20, plot_name="/bow_models/bow_binary_"):
    # Get the top most positive/negative coefficients
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    x_names = [feature_names[feature] for feature in top_coefficients]

    # Plot the coefficients
    plt.figure(figsize=(15, 5))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    plt.xticks(np.arange(0, 2 * top_features), x_names, rotation=30, ha='right')
    plt.ylabel("Coefficient Value")
    plt.title("Visualising the top %d features taken up by an SVM model" % top_features)
    to_save_filename = path + "/plots/" + plot_name + "top%d_coefficients.png" % top_features
    plt.savefig(to_save_filename)
    print("Coefficients' visualisation saved to %s\n" % to_save_filename)

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range


def grid_classifier(x_train, y_train, x_test, y_test, model, parameters,
                    make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    grid = GridSearchCV(estimator=model, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    if make_feature_analysis:
        plot_coefficients(classifier, feature_names, top_features, plot_name)
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# Method to print the header of the currently running model
def print_model_title(name):
    print("\n==================================================================")
    print('{:>20}'.format(name))
    print("==================================================================\n")


def linear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
               make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Linear SVM")
    C_range = get_regularization_params()
    parameters = {'C': C_range}
    linear_svm = LinearSVC(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, linear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def nonlinear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
                  make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Nonlinear SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'kernel': ['rbf'], 'C': C_range, 'gamma': gamma_range}
    nonlinear_svm = SVC(class_weight=class_ratio)
    grid_classifier(x_train, y_train, x_test, y_test, nonlinear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def logistic_regression_grid(x_train, y_train, x_test, y_test, class_ratio,
                        make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Logistic Regression")
    C_range = [0.001, 0.01, 0.1, 1, 10, 100]
    parameters = {'C': C_range}
    log_regr = LogisticRegression(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, log_regr, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def linear_svm(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Linear SVM")
    svm = LinearSVC(C=0.01, class_weight=class_ratio, penalty='l2')
    svm.fit(x_train, y_train)
    y_hat = svm.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/svm_model_bin.sav'
    # joblib.dump(svm, filename)


def logistic_regression(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Logistic Regression")
    regr = LogisticRegression(C=0.01, class_weight=class_ratio, penalty='l2')
    regr.fit(x_train, y_train)
    y_hat = regr.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
    # joblib.dump(regr, filename)


def random_forest(x_train, y_train, x_test, y_test, class_ratio='balanced'):
  print_model_title("Random Forest")
  rf = RandomForestClassifier(n_estimators=400, random_state=11)
  rf.fit(x_train, y_train)
  y_hat = rf.predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost")
  xgb_model =XGBClassifier(max_depth=6, n_estimators=1000)
  xgb_model .fit(x_train, y_train)
  y_hat = xgb_model .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def xg_boost_focal_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Focal")
  xgboster_focal = imb_xgb(special_objective='focal')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost_weighted_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Weighted")
  xgboster_focal = imb_xgb(special_objective='weighted')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def feature_selection(x_train, y_train, x_test, y_test):
    print("Feature selection with LinearSVC")
    model = LinearSVC(C=0.1, penalty='l2')
    rfe = RFE(model, 5)
    best_features_model = rfe.fit(x_train, y_train)
    y_hat = best_features_model.predict(x_test)
    print_statistics(y_test, y_hat)


def ensemble_stacked(x_train, y_train, x_test, y_test):
  print_model_title("Ensemble Stacked Classifiers")
  estimators = [ ('lr',LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),('xgb',XGBClassifier(max_depth=16, n_estimators=1000)),('svm_linear',LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),('rf', RandomForestClassifier(n_estimators=10, random_state=42))]
  from sklearn.ensemble import StackingClassifier
  clf = StackingClassifier(
      estimators=estimators )
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
def voting_classifiers(x_train, y_train, x_test, y_test,voting_type='hard'):
  print_model_title("Voting Classifier")
  estimators = [ ('lr',LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),('xgb',XGBClassifier(max_depth=16, n_estimators=1000)),('svm_linear',LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),('rf', RandomForestClassifier(n_estimators=10, random_state=42))]
  from sklearn.ensemble import StackingClassifier
  clf = VotingClassifier(
      estimators=estimators , voting=voting_type)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier
def Bagging_Classifier_LR(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier LR")
 
  clf =BaggingClassifier(base_estimator=LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def Bagging_Classifier_SVM(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier SVM")
 
  clf =BaggingClassifier(base_estimator=LinearSVC(C=0.01, class_weight='balanced', penalty='l2'),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


from sklearn.ensemble import GradientBoostingClassifier
def gradient_boosting(x_train, y_train, x_test, y_test):
  print_model_title("Gradient Boosting")
 
  clf =GradientBoostingClassifier(n_estimators=100, learning_rate=0.01,max_depth=30, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)


In [168]:
linear_svm_grid(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


          Linear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1

Accuracy: 0.717
Precision: 0.654
Recall: 0.717
F_score: 0.676

              precision    recall  f1-score   support

           0       0.77      0.90      0.83       531
           1       0.26      0.12      0.16       159

    accuracy                           0.72       690
   macro avg       0.51      0.51      0.50       690
weighted avg       0.65      0.72      0.68       690



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [169]:
nonlinear_svm_grid(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


       Nonlinear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Accuracy: 0.770
Precision: 0.592
Recall: 0.770
F_score: 0.669

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       531
           1       0.00      0.00      0.00       159

    accuracy                           0.77       690
   macro avg       0.38      0.50      0.43       690
weighted avg       0.59      0.77      0.67       690



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: 

In [170]:
logistic_regression_grid(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


 Logistic Regression



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Accuracy: 0.552
Precision: 0.671
Recall: 0.552
F_score: 0.587

              precision    recall  f1-score   support

           0       0.79      0.56      0.66       531
           1       0.26      0.51      0.34       159

    accuracy                           0.55       690
   macro avg       0.53      0.54      0.50       690
weighted avg       0.67      0.55      0.59       690



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [171]:
linear_svm(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.557
Precision: 0.669
Recall: 0.557
F_score: 0.591

              precision    recall  f1-score   support

           0       0.79      0.57      0.67       531
           1       0.26      0.50      0.34       159

    accuracy                           0.56       690
   macro avg       0.53      0.54      0.50       690
weighted avg       0.67      0.56      0.59       690



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [172]:
logistic_regression(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.577
Precision: 0.692
Recall: 0.577
F_score: 0.610

              precision    recall  f1-score   support

           0       0.81      0.58      0.68       531
           1       0.28      0.55      0.38       159

    accuracy                           0.58       690
   macro avg       0.55      0.57      0.53       690
weighted avg       0.69      0.58      0.61       690



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [176]:
random_forest(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


       Random Forest



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Accuracy: 0.759
Precision: 0.645
Recall: 0.759
F_score: 0.672

              precision    recall  f1-score   support

           0       0.77      0.98      0.86       531
           1       0.23      0.02      0.03       159

    accuracy                           0.76       690
   macro avg       0.50      0.50      0.45       690
weighted avg       0.65      0.76      0.67       690



In [174]:
xg_boost(x_train,y_train.astype(int),x_test,y_test.astype(int))


             XGBoost



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.732
Precision: 0.661
Recall: 0.732
F_score: 0.683

              precision    recall  f1-score   support

           0       0.77      0.92      0.84       531
           1       0.28      0.11      0.16       159

    accuracy                           0.73       690
   macro avg       0.53      0.51      0.50       690
weighted avg       0.66      0.73      0.68       690



In [175]:
ensemble_stacked(x_train,y_train.astype(int),x_test,y_test.astype(int))


Ensemble Stacked Classifiers



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.771
Precision: 0.747
Recall: 0.771
F_score: 0.675

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       531
           1       0.67      0.01      0.02       159

    accuracy                           0.77       690
   macro avg       0.72      0.51      0.45       690
weighted avg       0.75      0.77      0.68       690



In [177]:
voting_classifiers(x_train,y_train.astype(int),x_test,y_test.astype(int))


   Voting Classifier



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.748
Precision: 0.681
Recall: 0.748
F_score: 0.695

              precision    recall  f1-score   support

           0       0.78      0.94      0.85       531
           1       0.35      0.11      0.17       159

    accuracy                           0.75       690
   macro avg       0.57      0.53      0.51       690
weighted avg       0.68      0.75      0.69       690



In [178]:
Bagging_Classifier_LR(x_train,y_train.astype(int),x_test,y_test.astype(int))


Bagging Calssifier LR



/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.584
Precision: 0.684
Recall: 0.584
F_score: 0.615

              precision    recall  f1-score   support

           0       0.81      0.61      0.69       531
           1       0.28      0.51      0.36       159

    accuracy                           0.58       690
   macro avg       0.54      0.56      0.53       690
weighted avg       0.68      0.58      0.62       690



In [179]:
Bagging_Classifier_SVM(x_train,y_train.astype(int),x_test,y_test.astype(int))


Bagging Calssifier SVM



/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Accuracy: 0.581
Precision: 0.669
Recall: 0.581
F_score: 0.611

              precision    recall  f1-score   support

           0       0.79      0.62      0.69       531
           1       0.26      0.45      0.33       159

    accuracy                           0.58       690
   macro avg       0.53      0.54      0.51       690
weighted avg       0.67      0.58      0.61       690



In [180]:
gradient_boosting(x_train,y_train.astype(int),x_test,y_test.astype(int))


   Gradient Boosting



/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.684
Precision: 0.655
Recall: 0.684
F_score: 0.668

              precision    recall  f1-score   support

           0       0.78      0.83      0.80       531
           1       0.26      0.19      0.22       159

    accuracy                           0.68       690
   macro avg       0.52      0.51      0.51       690
weighted avg       0.66      0.68      0.67       690



In [181]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1,1),
    dtype=np.float32,
    max_features=5000
)


# Character Stemmer
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    dtype=np.float32,
    max_features=4000
)

word_vectorizer.fit(result['text'])
char_vectorizer.fit(result['text'])


TfidfVectorizer(analyzer='char', dtype=<class 'numpy.float32'>,
                max_features=4000, ngram_range=(1, 4), strip_accents='unicode',
                sublinear_tf=True)

In [182]:
train_word_features = word_vectorizer.transform(result['text'])
train_char_features = char_vectorizer.transform(result['text'])

In [183]:
test_word_features = word_vectorizer.transform(result_test['text'])
test_char_features = char_vectorizer.transform(result_test['text'])

In [185]:
from scipy.sparse import hstack, csr_matrix


In [190]:
train_features = hstack([
    train_char_features,
    train_word_features
    ,csr_matrix(np.array(x_train.to_numpy(), dtype=float))], 'csr'
)

In [191]:
test_features = hstack([
    test_char_features,
    test_word_features
    ,csr_matrix(np.array(x_test.to_numpy(), dtype=float))], 'csr'
)

In [192]:
linear_svm_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


          Linear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1

Accuracy: 0.696
Precision: 0.698
Recall: 0.696
F_score: 0.697

              precision    recall  f1-score   support

           0       0.80      0.80      0.80       531
           1       0.34      0.35      0.35       159

    accuracy                           0.70       690
   macro avg       0.57      0.58      0.57       690
weighted avg       0.70      0.70      0.70       690



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [193]:
nonlinear_svm_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


       Nonlinear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Accuracy: 0.770
Precision: 0.592
Recall: 0.770
F_score: 0.669

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       531
           1       0.00      0.00      0.00       159

    accuracy                           0.77       690
   macro avg       0.38      0.50      0.43       690
weighted avg       0.59      0.77      0.67       690



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: 

In [194]:
logistic_regression_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Accuracy: 0.691
Precision: 0.707
Recall: 0.691
F_score: 0.698

              precision    recall  f1-score   support

           0       0.81      0.78      0.79       531
           1       0.35      0.40      0.38       159

    accuracy                           0.69       690
   macro avg       0.58      0.59      0.59       690
weighted avg       0.71      0.69      0.70       690



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [195]:
linear_svm(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



          Linear SVM

Accuracy: 0.635
Precision: 0.705
Recall: 0.635
F_score: 0.659

              precision    recall  f1-score   support

           0       0.82      0.67      0.74       531
           1       0.32      0.51      0.39       159

    accuracy                           0.63       690
   macro avg       0.57      0.59      0.57       690
weighted avg       0.70      0.63      0.66       690



In [196]:
logistic_regression(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.588
Precision: 0.700
Recall: 0.588
F_score: 0.620



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.82      0.60      0.69       531
           1       0.30      0.57      0.39       159

    accuracy                           0.59       690
   macro avg       0.56      0.58      0.54       690
weighted avg       0.70      0.59      0.62       690



In [197]:
random_forest(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')



       Random Forest



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Accuracy: 0.762
Precision: 0.643
Recall: 0.762
F_score: 0.671

              precision    recall  f1-score   support

           0       0.77      0.99      0.86       531
           1       0.22      0.01      0.02       159

    accuracy                           0.76       690
   macro avg       0.50      0.50      0.44       690
weighted avg       0.64      0.76      0.67       690



In [198]:
xg_boost(train_features,y_train.astype(int),test_features,y_test.astype(int))



             XGBoost



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.745
Precision: 0.663
Recall: 0.745
F_score: 0.682

              precision    recall  f1-score   support

           0       0.77      0.95      0.85       531
           1       0.29      0.08      0.12       159

    accuracy                           0.74       690
   macro avg       0.53      0.51      0.49       690
weighted avg       0.66      0.74      0.68       690



In [199]:
ensemble_stacked(train_features,y_train.astype(int),test_features,y_test.astype(int))



Ensemble Stacked Classifiers



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.768
Precision: 0.702
Recall: 0.768
F_score: 0.684

              precision    recall  f1-score   support

           0       0.77      0.99      0.87       531
           1       0.46      0.04      0.07       159

    accuracy                           0.77       690
   macro avg       0.62      0.51      0.47       690
weighted avg       0.70      0.77      0.68       690



In [200]:
voting_classifiers(train_features,y_train.astype(int),test_features,y_test.astype(int))



   Voting Classifier



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.754
Precision: 0.676
Recall: 0.754
F_score: 0.688

              precision    recall  f1-score   support

           0       0.78      0.96      0.86       531
           1       0.34      0.08      0.12       159

    accuracy                           0.75       690
   macro avg       0.56      0.52      0.49       690
weighted avg       0.68      0.75      0.69       690



In [201]:
Bagging_Classifier_LR(train_features,y_train.astype(int),test_features,y_test.astype(int))



Bagging Calssifier LR



/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.590
Precision: 0.682
Recall: 0.590
F_score: 0.620

              precision    recall  f1-score   support

           0       0.80      0.62      0.70       531
           1       0.28      0.49      0.36       159

    accuracy                           0.59       690
   macro avg       0.54      0.56      0.53       690
weighted avg       0.68      0.59      0.62       690



In [202]:
Bagging_Classifier_SVM(train_features,y_train.astype(int),test_features,y_test.astype(int))



Bagging Calssifier SVM



/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.648
Precision: 0.688
Recall: 0.648
F_score: 0.664

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       531
           1       0.30      0.41      0.35       159

    accuracy                           0.65       690
   macro avg       0.55      0.56      0.55       690
weighted avg       0.69      0.65      0.66       690



In [203]:
gradient_boosting(train_features,y_train.astype(int),test_features,y_test.astype(int))


   Gradient Boosting



/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.752
Precision: 0.629
Recall: 0.752
F_score: 0.668

              precision    recall  f1-score   support

           0       0.77      0.97      0.86       531
           1       0.17      0.02      0.03       159

    accuracy                           0.75       690
   macro avg       0.47      0.50      0.45       690
weighted avg       0.63      0.75      0.67       690

